In [20]:
import Bio.Seq
import Bio.SeqUtils as SU
from Bio.Seq import Seq
from Bio.SeqUtils import GC

from Bio import SeqIO

record_orig = ""

with open("01.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        record_orig = record
        # print(record)

gc_orig = Bio.SeqUtils.gc_fraction(record_orig)
print(f'Original record length: {record_orig.__len__()}')

print(f'Original GC: {gc_orig}')

current_sub = ""
current_max_insertion = ""
border_left = 0
border_right = 0
# Порог ошибки, где начинается вставка
e = 0.05
min_size = 1000
step_size = 100

for i in range(0, len(record_orig), step_size):
    print(i)
    for j in range(i+min_size, len(record_orig)+1, step_size):
        current_sub = record_orig[i:j]
        if (abs(gc_orig-SU.gc_fraction(current_sub)) > e):
            if (len(current_max_insertion) < len(current_sub)):
                current_max_insertion = current_sub
                border_left = i
                border_right = j


SyntaxError: f-string: empty expression not allowed (951493013.py, line 18)

In [ ]:
print(f'Insertion length: {current_max_insertion.__len__()}')
print(f'Borders: left - {border_left}; right - {border_right}')
print(f'Insertion GC: {SU.gc_fraction(current_max_insertion)}')

Insertion length: 32100
Borders: left - 0; right - 32100
Insertion GC: 0.27763239875389406
